In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm.list_resources())

deviceIDNs = {}

for device in rm.list_resources():
    _device = rm.open_resource(device)
    _IDN = _device.query('*IDN?')
    deviceIDNs[device] = _IDN

for key,value in deviceIDNs.items():
    if 'SO-X' in value:
        scopeResource = key
        break

scope = rm.open_resource(scopeResource)
scope.timeout = 10000
print(scope.query('*IDN?'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def waveformCapture(channel, timestampOffset = 0):
    #Setup data export
    scope.write(":WAVeform:FORMat WORD")                    #16 bit word format
    scope.write(":WAVeform:BYTeorder LSBFirst")             #Explicitly set this to avoid confusion
    scope.write(":WAVeform:UNSigned 0")                     #Explicitly set this to avoid confusion
    scope.write(f":WAVeform:SOURce CHANnel {channel}")      #Set waveform source
    scope.write(":WAVeform:POINts:MODE NORMal")             #Normal / Max

    #Get preamble data and parse it
    preamble = scope.query(":WAVeform:PREamble?").split(',')

    acqMode        = int(preamble[1])                       #Scope acquisition mode
    datapoints     = int(preamble[2])                       #Get number of acqusition points
    xIncrement     = float(preamble[4])                     #Time difference between data points
    xOrigin        = float(preamble[5])                     #Always the first data point in memory
    xReference     = int(preamble[6])                       #Specifies the data point associated with x-origin
    yIncrement     = float(preamble[7])                     #Voltage difference between data points
    yOrigin        = float(preamble[8])                     #Voltage at center screen
    yReference     = int(preamble[9])                       #Sepcifies the data point associated with y-origin

    #Create an arry of time values using the preamble data, convert it to a dataframe
    waveformTime = ((np.linspace(0, datapoints-1, datapoints) - xReference) * xIncrement ) + xOrigin + timestampOffset
    waveformTimeDataFrame = pd.DataFrame(waveformTime, columns = ['Timestamp'])

    #Get the waveform data, convert it to a dataframe, scale it
    waveformData = scope.query_binary_values(':WAVeform:SOURce CHANnel 1' + ';DATA?', "h", False)  #Returns a List
    waveformDataFrame = pd.DataFrame(waveformData, columns = ['Raw Data'])
    waveformDataFrame['Scaled Data'] = ((waveformDataFrame['Raw Data'] - yReference) * yIncrement) + yOrigin

    #Join the time dataframe to the voltage dataframe
    waveformOutput = waveformTimeDataFrame.join(waveformDataFrame)

    return(waveformOutput)

def waveformPlotter(acqDataframe, figsizeX = 10):
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html
    plt.close("all")

    waveformPlot = acqDataframe.plot(x = 'Timestamp', y = 'Scaled Data', title = 'Waveform Data', xlabel = 'Time (Seconds)', ylabel = 'Voltage (Volts)', \
        figsize = (figsizeX,8) , grid = True)

    return(waveformPlot)

In [ ]:
#Single Capture
samplePlot = waveformPlotter(waveformCapture(1))   
samplePlot.get_figure().savefig("output.png")

In [ ]:
#Segmented Capture | With Time Offset
segmentCount = int(scope.query(":WAVeform:SEGMented:COUNt?"))

segmentTimetags = {}
waveformDataframeList = []

for n in range(1, segmentCount + 1, 1):   
    scope.write(f":ACQuire:SEGMented:INDex {n}")
    segmentTimetags[n] = float(scope.query(":WAVeform:SEGMented:TTAG?"))  # Get time tag of segment n ; always get time tags
    
for key,value in segmentTimetags.items():
    scope.write(f":ACQuire:SEGMented:INDex {key}")
    capture = waveformCapture(1, value)
    capture['Segment Number'] = key
    waveformDataframeList.append(capture)

allSegmentOutput = pd.concat(waveformDataframeList)
waveformPlotter(allSegmentOutput, figsizeX = 26)

In [ ]:
#Segmented Capture | Without Time Offset
segmentCount = int(scope.query(":WAVeform:SEGMented:COUNt?"))

segmentTimetags = {}
timespanList = []
waveformDataframeList = []

for n in range(1, segmentCount + 1, 1):   
    scope.write(f":ACQuire:SEGMented:INDex {n}")
    segmentTimetags[n] = float(scope.query(":WAVeform:SEGMented:TTAG?"))  # Get time tag of segment n ; always get time tags
    
for key,value in segmentTimetags.items():
    scope.write(f":ACQuire:SEGMented:INDex {key}")
    if key == 1:
        capture = waveformCapture(1)
        capture['Segment Number'] = key
        waveformDataframeList.append(capture)
        timespan = capture['Timestamp'].max() - capture['Timestamp'].min()
        triggerOffset = capture['Timestamp'].min()
    else:
        capture = waveformCapture(1, timespan)
        capture['Segment Number'] = key
        waveformDataframeList.append(capture)
        timespan = timespan + capture['Timestamp'].max() - capture['Timestamp'].min()
    timespanList.append(timespan)
        

allSegmentOutput = pd.concat(waveformDataframeList)
samplePlot = waveformPlotter(allSegmentOutput, figsizeX = 26)

for timespan in timespanList:
    samplePlot.axvline(x = timespan + triggerOffset, color = 'r')